### 2.1 
Diagram of the baseline model: **See Attachement: Q2_1_2.pdf**

In [2]:
# from IPython.display import IFrame
# IFrame("diagram.pdf", width=600, height=300)

In [ ]:
# get more RAM 
#d=[]
#while(1):
#    d.append('1')

In [2]:
! git clone https://github.com/williamFalcon/squad.git

Cloning into 'squad'...
remote: Enumerating objects: 47, done.
remote: Total 47 (delta 0), reused 0 (delta 0), pack-reused 47
Unpacking objects: 100% (47/47), done.


In [3]:
! cd squad/ && git pull

Already up to date.


In [4]:
! pip install ujson
! pip install tensorboardX

     |████████████████████████████████| 194kB 4.8MB/s eta 0:00:01
  Created wheel for ujson: filename=ujson-1.35-cp36-cp36m-linux_x86_64.whl size=68037 sha256=010a67d8e753da2ae84e424c417692f5334a90f77ff88a722668355b5c3cc19a
  Stored in directory: /root/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built ujson
     |████████████████████████████████| 194kB 4.9MB/s 


In [5]:
! cd squad/ && python setup.py

glove.840B.300d.zip: 2.18GB [16:55, 2.14MB/s]                
Unzipping GloVe word vectors...
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
Pre-processing train examples...
100% 442/442 [01:32<00:00,  1.82it/s]
130319 questions in total
Pre-processing word vectors...
2196018it [03:03, 11964.64it/s]                 
88744 / 107454 tokens have corresponding word embedding vector
Pre-processing char vectors...
1374 tokens have corresponding char embedding vector
Pre-processing dev examples...
100% 16/16 [00:04<00:00,  3.82it/s]
6078 questions in total
Converting train examples to indices...
130319it [00:42, 3079.60it/s]
tcmalloc: large alloc 3326009344 bytes == 0x138fb2000 @  0x7f5f8c7321e7 0x7f5f8a211f71 0x7f5f8a27555d 0x7f5f8a278e28 0x7f5f8a2793e5 0x7f5f8a30

In [ ]:
import numpy.random as random
import numpy as np
seed = 224
random.seed(seed)
import torch
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as sched
import torch.utils.data as data

from squad.util import collate_fn, SQuAD
from collections import OrderedDict
from json import dumps
from tqdm import tqdm
from ujson import load as json_load

from squad import layers, util
import os

In [1]:
# add self.rnn.flatten_parameters() under the forward functionin layers RNNEncoder
# change the code in the layers "from util import masked_softmax"

# change the concatnation to element-wise multiplication.
# change the RNNEncoder in layers.py:
        # this_hidden_size = int(len(x[0][0]) / 2)
        # x = (x[:,:,:this_hidden_size] * x[:,:,-this_hidden_size:])
# change the BiDAF in models.py:
        # att and mod: deduce by half
# change the BiDAFOutput in layers.py:
        # deduce by half


### Train the baseline model 

In [9]:
! cd squad/ && python train.py -n baseline_multiply --batch_size 64

[12.03.19 20:49:55] Args: {
    "batch_size": 64,
    "char_emb_file": "./data/char_emb.json",
    "dev_eval_file": "./data/dev_eval.json",
    "dev_record_file": "./data/dev.npz",
    "drop_prob": 0.2,
    "ema_decay": 0.999,
    "eval_steps": 50000,
    "gpu_ids": [
        0
    ],
    "hidden_size": 100,
    "l2_wd": 0,
    "load_path": null,
    "lr": 0.5,
    "max_ans_len": 15,
    "max_checkpoints": 5,
    "max_grad_norm": 5.0,
    "maximize_metric": true,
    "metric_name": "F1",
    "name": "baseline_multiply",
    "num_epochs": 30,
    "num_visuals": 10,
    "num_workers": 4,
    "save_dir": "./save/train/baseline_multiply-01",
    "seed": 224,
    "test_eval_file": "./data/test_eval.json",
    "test_record_file": "./data/test.npz",
    "train_eval_file": "./data/train_eval.json",
    "train_record_file": "./data/train.npz",
    "use_squad_v2": true,
    "word_emb_file": "./data/word_emb.json"
}
[12.03.19 20:49:55] Using random seed 224...
[12.03.19 20:49:55] Loading embeddin

In [ ]:
!pwd

/content


In [10]:
# elementwise multiplication model
!md5sum "/content/squad/save/train/baseline_multiply-01/best.pth.tar"

224e91451c044b322c8fee7335f9f0e8  /content/squad/save/train/baseline_multiply-01/best.pth.tar


In [ ]:
# baseline model
!md5sum "/content/squad/save/train/baseline-01/best.pth.tar"

5e64bca710edea8927ac3d16adc917aa  /content/squad/save/train/baseline-01/best.pth.tar


In [ ]:
from google.colab import files
files.download( "" ) 

In [ ]:
!pwd

/content


In [ ]:
train_path = os.path.join(DIR, 'squad/data/train.npz')
train_loader = data.DataLoader(SQuAD(train_path),
                               batch_size=64,
                               shuffle=True,
                               drop_last=True,
                               collate_fn=collate_fn)
dev_path = os.path.join(DIR, 'squad/data/dev.npz')
dev_loader = data.DataLoader(SQuAD(dev_path),
                               batch_size=64,
                               shuffle=True,
                               drop_last=True,
                               collate_fn=collate_fn)
test_path = os.path.join(DIR, 'squad/data/test.npz')
test_loader = data.DataLoader(SQuAD(test_path),
                               batch_size=64,
                               shuffle=True,
                               drop_last=True,
                               collate_fn=collate_fn)

In [ ]:
def estimate_alpha(data_loader):
  no_answer = 0
  total = 0
  for batch in data_loader:
    cw_idxs, cc_idxs, qw_idxs, qc_idxs, y1, y2, ids = batch
    no_answer += sum(y1 == 0).item()
    total += y1.size(-1)    
  return no_answer/total

In [ ]:
print("alpha: ",estimate_alpha(train_loader))


alpha:  0.33364378078817736


In [ ]:
alpha = estimate_alpha(train_loader)


In [ ]:

## implement random QA model
## load dataset 
## find F1 and EM on Dev Set

dev_dataset = SQuAD(dev_path)

f1_list = []
em_list = []

for i in range(100):
  f1 = 0.0
  em = 0.0
  for cw_idxs, cc_idxs, qw_idxs, qc_idxs, y1, y2, ids in dev_dataset:
    real_len = sum(cw_idxs != 0).item()
    test = np.random.rand()
    if test <= alpha:
      if y1.item() == 0:
        f1 += 1
        em += 1
    else:
      if y1.item() != 0 and y2.item() != 0:
        while True:
          start_idx = np.random.randint(0,real_len)
          end_idx = np.random.randint(0,real_len)
          if end_idx >= start_idx:
            preds_ids = cw_idxs[start_idx:end_idx+1].tolist()
            preds = " ".join([str(id) for id in preds_ids])
            answer_ids = cw_idxs[y1.item():y2.item()+1].tolist()
            answer = " ".join([str(id) for id in answer_ids])
            f1 += util.compute_f1(answer,preds)
            em += util.compute_em(answer,preds)
            break

  f1_score = f1/len(dev_dataset)
  em_score = em/len(dev_dataset)  
  f1_list.append(f1_score)
  em_list.append(em_score)

In [ ]:
print("mean F1: ",np.mean(f1_list))
print("F1 std: ", np.std(f1_list))
print("mean EM: ",np.mean(em_list))
print("EN std: ", np.std(em_list))

mean F1:  0.1941990820614662
F1 std:  0.0043236481269492725
mean EM:  0.1739539573180978
EN std:  0.004376310750118215
